## Carga y filtrado de imagenes

Pasos generales:
* Subir el Archivo de datos a su drive (Si es necesario)
* Cambiar la direccion a su direccion en su drive (Si es necesario)

In [ ]:
import os
import zipfile

# Definir el nombre del archivo ZIP y la ruta en Google Drive
zip_file = "/content/drive/MyDrive/Ciencia de datos/Proyecto/Datosp.zip"

# Definir el directorio de destino donde se descomprimirá el archivo
destination_dir = "/content/drive/MyDrive/Ciencia de datos/Proyecto"

# Descomprimir el archivo ZIP
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination_dir)

# Verificar que el archivo se haya descomprimido correctamente
print(os.listdir(destination_dir))

* Creamos una copia de seguridad del archivo para evitar futuros problemas

In [ ]:
import shutil
import os

# Definir la ruta del directorio original
original_dir = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"

# Definir la nueva ruta con el nombre cambiado
new_dir = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data_original"

# Copiar el directorio y renombrarlo
shutil.copytree(original_dir, new_dir) # Use shutil.copytree() for directories

# Verificar que el directorio se haya copiado correctamente
if os.path.exists(new_dir):
    print(f"Directorio copiado y renombrado a: {new_dir}")
else:
    print("La copia no se realizó correctamente.")

Algunas imágenes del archivo original (vehiculos_data) parecen estar dañadas o en un formato no compatible, lo que generaba errores. Por esta razón, se procedió a eliminar las imágenes que causaban dichos problemas mediante la siguiente funcion.

* Funcion para filtar las imagenes que producen errores

In [ ]:
import tensorflow as tf

def load_and_filter_image(image_path):
    try:
        # Leer el archivo de imagen
        img = tf.io.read_file(image_path)
        # Decodificar la imagen a un formato usable
        img = tf.image.decode_image(img)
        return img
    except tf.errors.InvalidArgumentError as e:
        # Manejar errores de carga y decodificación
        print(f"Error al cargar la imagen: {image_path}")
        return None

* Aplicando funcion para resolver los errores (Este codigo solo funciona una vez, no correra amenos de que la base presente los problemas iniciales)

In [ ]:
# Definir el directorio raíz que contiene las imágenes
root_directory = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"

# Recorrer todos los archivos en el directorio y subdirectorios
for root, dirs, files in os.walk(root_directory):
    for filename in files:
        file_path = os.path.join(root, filename)
        # Filtrar solo archivos de imagen con extensiones válidas
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
            # Intentar cargar la imagen
            image = load_and_filter_image(file_path)
            # Si la imagen no se pudo cargar, eliminar el archivo
            if image is None:
                os.remove(file_path)
                print(f"Archivo eliminado: {file_path}")

* Conteo de archivos para verificar cuantos fueron eliminados, etc.

In [ ]:
import os

# Especificar la ruta de la carpeta original (antes de eliminar archivos)
root_directory = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"
folder_path = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data_original"

# Contar recursivamente todos los archivos y carpetas en la carpeta principal
element_count = 0
for root, dirs, files in os.walk(folder_path):
    element_count += len(dirs) + len(files)

# Mostrar el número total de elementos (archivos y carpetas) antes de eliminar
print(f"Número total de elementos en la carpeta '{folder_path}' y sus subdirectorios: {element_count}")

# Contar recursivamente todos los archivos y carpetas en la carpeta filtrada
element_count2 = 0
for root, dirs, files in os.walk(root_directory):
    element_count2 += len(dirs) + len(files)

# Mostrar el número total de elementos después de filtrar
print(f"Número total de elementos en la carpeta '{root_directory}' y sus subdirectorios: {element_count2}")

# Cálculos de archivos eliminados
sinfiltrar = element_count
filtrados = element_count2
eliminados = sinfiltrar - filtrados

# Mostrar resultados finales
print(f"Número total de archivos eliminados que provocaban un error: {eliminados}")

* Listo este es el proceso incial mediante el cual se se cargaron los datos, se descomprimieron, se creo una copia de seguridad, y se filtraron los datos que producian algun tipo de error ala hora de subir los datos para el modelo.

## Carga de datos y caracteristicas

* Cargar los datos al modelo

Asegurarse de que esta conectado al Drive para que no precenten errores en esta primera parte

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Especificar el directorio que contiene las imágenes
directory = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"

# Cargar los datos como root_directory
data = keras.utils.image_dataset_from_directory(
    directory,               # Directorio que contiene las imágenes
    labels="inferred",       # Inferir las etiquetas de los nombres de las carpetas
    label_mode="int",        # Usar enteros como modo de etiquetas
    color_mode="rgb",        # Cargar imágenes en modo RGB
    batch_size=6,            # Tamaño del lote
    shuffle=True,            # Aleatorizar los datos
    seed=1234                # Semilla para la aleatorización
)

* Caracteristicas del conjunto de datos

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Mostrar las características de las clases en el conjunto de datos
print(data.class_names)

# Inicializar un diccionario para contar observaciones por categoría
observations_per_category = {class_name: 0 for class_name in data.class_names}

# Contar el número de observaciones por categoría
for images, labels in data:
    labels = labels.numpy()  # Convertir etiquetas a un formato NumPy
    # Contar etiquetas únicas y sus ocurrencias
    unique, counts = np.unique(labels, return_counts=True)

    # Actualizar el diccionario con los conteos
    for idx, count in zip(unique, counts):
        observations_per_category[data.class_names[idx]] += count

# Imprimir el número de categorías en el conjunto de datos
print(f"Número de categorías: {len(np.unique(data.class_names))}")
# Mostrar el número de observaciones por categoría
print("Número de observaciones por categoría:")
for class_name, count in observations_per_category.items():
    print(f"  {class_name}: {count}")

* Ejemplos por categoria

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Configurar la figura para visualizar las imágenes
plt.figure(figsize=(10, 10))

# Inicializar un conjunto para seguir las etiquetas que ya se han mostrado
seen_labels = set()

# Inicializar un contador de imágenes
image_count = 0

# Iterar sobre el conjunto de datos para encontrar una imagen por categoría
for images, labels in data:
    for j in range(images.shape[0]):
        label = labels[j].numpy()  # Obtener la etiqueta de la imagen

        # Si la etiqueta ya ha sido vista, continuar
        if label in seen_labels:
            continue

        # Añadir la etiqueta al conjunto de etiquetas vistas
        seen_labels.add(label)

        # Obtener una sola imagen del lote
        single_image = images[j].numpy().astype("uint8")  # Convertir la imagen a uint8

        # Ajustar la posición del subplot, incrementando el contador
        plt.subplot(5, 4, image_count + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(single_image)  # Mostrar la imagen
        plt.xlabel(data.class_names[label])  # Etiqueta correspondiente

        # Incrementar el contador de imágenes
        image_count += 1

        # Si se han mostrado 20 imágenes, salir del bucle
        if image_count >= 20:
            break
    if image_count >= 20:
        break  # Salir del bucle principal si ya se mostraron 20 imágenes

plt.show()

# Modelos

## Modelo MLP

* Carga de datos

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Especificar el directorio que contiene las imágenes
#directory = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"
directory = "C:/Users/cesar/OneDrive/Escritorio/vehicle_data"

# Cargar los datos como root_directory
data = keras.utils.image_dataset_from_directory(
    directory,               # Directorio que contiene las imágenes
    labels="inferred",       # Inferir las etiquetas de los nombres de las carpetas
    label_mode="int",        # Usar enteros como modo de etiquetas
    color_mode="rgb",        # Cargar imágenes en modo RGB
    batch_size=6,            # Tamaño del lote
    shuffle=True,            # Aleatorizar los datos
    seed=1234                # Semilla para la aleatorización
)


* Division de la base y transformacion de los datos

In [ ]:
# Importar las librerías necesarias
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# Convertir datos a numpy arrays
# (esto puede variar dependiendo de cómo cargaste los datos)
images, labels = [], []  # Listas para almacenar imágenes y etiquetas

# Recorrer los lotes de datos y agregar a las listas
for image_batch, label_batch in data:
    images.extend(image_batch.numpy())  # Convertir y agregar imágenes
    labels.extend(label_batch.numpy())   # Convertir y agregar etiquetas

# Convertir listas a arrays de numpy
images = np.array(images)
labels = np.array(labels)

# Estratificación y división de datos
# X contendrá las imágenes y y contendrá las etiquetas
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.3, random_state=42, stratify=labels
)

# Dividir el conjunto de prueba en validación y prueba
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

# Mostrar la cantidad de datos en cada conjunto
print(len(y_train), len(y_val), len(y_test))

# Normalizar los píxeles entre 0 y 1
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Codificación one-hot para las etiquetas
num_classes = len(data.class_names)  # Número de clases
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Aplanar las imágenes en 1D
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

* Verificar conteos por division, etc.

In [ ]:
# Imprimir las formas de los conjuntos de datos
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de validación:", X_val.shape)

# Mostrar los nombres de las clases en el conjunto de datos
print("Nombres de las clases:", data.class_names)
print("Número de categorías:", len(data.class_names))

# Imprimir los valores mínimo y máximo en el conjunto de entrenamiento
print("Valor mínimo en X_train:", X_train.min())
print("Valor máximo en X_train:", X_train.max())

* Entrenamiento y modelo

In [ ]:
# Importar las librerías necesarias
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo secuencial
model = keras.models.Sequential()

# Añadir capas densas con regularización L2
model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train.shape[1],)))  # Capa de entrada
model.add(layers.Dense(144, activation='relu', kernel_regularizer=regularizers.l2(0.001)))  # Capa oculta
model.add(layers.Dense(81, activation='relu', kernel_regularizer=regularizers.l2(0.001)))  # Otra capa oculta
model.add(layers.Dense(49, activation='relu', kernel_regularizer=regularizers.l2(0.001)))  # Capa oculta más pequeña

# Capa de salida
model.add(layers.Dense(num_classes, activation='softmax'))  # Usar num_classes para el número de salidas

# Definir el optimizador
optimizer = Adam(learning_rate=0.0001)

# Compilar el modelo
model.compile(optimizer=optimizer,  # Optimizador Adam
              loss='categorical_crossentropy',  # Función de pérdida para clasificación
              metrics=['accuracy'])  # Métrica a seguir

# Definir el callback de EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Métrica a monitorear
    patience=10,  # Número de épocas a esperar sin mejora
    restore_best_weights=True  # Restaurar los mejores pesos al final
)

# Entrenar el modelo
history = model.fit(X_train, y_train,  # Datos de entrenamiento
                    epochs=100,  # Número máximo de épocas
                    batch_size=32,  # Tamaño del lote
                    validation_data=(X_val, y_val),  # Datos de validación
                    callbacks=[early_stopping])  # Callback de early stopping

* Evaluacion del modelo

In [ ]:
basic_model_eval = model.evaluate(X_test, y_test)
print(f"Modelo básico: error y accuracy {basic_model_eval}")

* Guardar el modelo

In [ ]:
# Guardar
#model.save('model.h5')
# Volver a cargar en el ambiente un modelo guardado
#from tensorflow.keras.models import load_model
#model = load_model('model.h5')

* Evaluacion grafica

In [ ]:
# Importar las librerías necesarias
import matplotlib.pyplot as plt  # Para graficar
import tensorflow as tf          # Para manejar el historial del entrenamiento

# Acceder a la precisión de entrenamiento y validación desde el historial
train_accuracy = history.history['accuracy']    # Precisión de entrenamiento
val_accuracy = history.history['val_accuracy']  # Precisión de validación

# Acceder a la pérdida de entrenamiento y validación desde el historial
train_loss = history.history['loss']         # Pérdida de entrenamiento
val_loss = history.history['val_loss']       # Pérdida de validación

# Configuración de la figura con 1 fila y 2 columnas
fig, axs = plt.subplots(1, 2, figsize=(16, 6))  # Ajusta el tamaño según lo necesites

# Graficar precisión de entrenamiento y validación
axs[0].plot(train_accuracy, label='Entrenamiento', color="blue")    # Precisión de entrenamiento
axs[0].plot(val_accuracy, label='Validación', color="red")          # Precisión de validación
axs[0].set_title('Accuracy de entrenamiento por epoch')  # Título del gráfico
axs[0].set_xlabel('Epoch')                               # Etiqueta del eje X
axs[0].set_ylabel('Precisión')                           # Etiqueta del eje Y
axs[0].set_ylim(0, 1)
axs[0].legend()                                          # Mostrar la leyenda
axs[0].grid(True)                                        # Activar la cuadrícula

# Graficar pérdida de entrenamiento y validación
axs[1].plot(train_loss, label='Entrenamiento', color="blue")    # Pérdida de entrenamiento
axs[1].plot(val_loss, label='Validación', color="red")          # Pérdida de validación
axs[1].set_title('Error de entrenamiento por epoch')   # Título del gráfico
axs[1].set_xlabel('Epoch')                             # Etiqueta del eje X
axs[1].set_ylabel('Error')                             # Etiqueta del eje Y
axs[1].legend()                                        # Mostrar la leyenda
axs[1].grid(True)                                      # Activar la cuadrícula

plt.tight_layout()                                     # Ajustar el diseño para evitar solapamientos
plt.show()                                             # Mostrar los gráficos

* Mostrar resultados de imágenes y sus clasificaciones

In [ ]:
# Importar las librerías necesarias
import numpy as np           # Para operaciones numéricas y manipulación de arrays
import matplotlib.pyplot as plt  # Para visualización de imágenes

# Mostrar 10 imágenes con sus clasificaciones reales y predichas
n = 10  # Número de imágenes a mostrar
fig, ax = plt.subplots(1, n, figsize=(9, 9))

for i in range(n):
    # Obtener la clasificación real y la predicción del modelo
    real = np.argmax(y_test[i])  # Clasificación real de la imagen
    prediction = model.predict(X_test[i].reshape(1, -1))  # Predicción del modelo
    prediction = np.argmax(prediction)  # Obtener el índice de la clase predicha

    # Calcular las dimensiones correctas de la imagen
    image_size = int(np.sqrt(X_test[i].shape[0] / 3))  # Suponiendo que las imágenes son cuadradas y tienen 3 canales de color

    # Reconfigurar los datos de la imagen con el tamaño calculado
    ax[i].imshow(X_test[i].reshape(image_size, image_size, 3))  # Mostrar la imagen
    ax[i].axis("off")  # Ocultar los ejes
    ax[i].set_title(f"Real: {data.class_names[real]}, \n Clasificado: {data.class_names[prediction]}", fontsize=5)  # Título con las clasificaciones

plt.show()  # Mostrar la figura con las imágenes

* Matriz de confucion

In [ ]:
# Importar las librerías necesarias
import numpy as np                         # Para operaciones numéricas y manipulación de arrays
from sklearn.metrics import confusion_matrix  # Para generar la matriz de confusión
import seaborn as sns                      # Para visualizar la matriz de confusión
import matplotlib.pyplot as plt            # Para visualización de gráficos

# Calcular las clasificaciones reales y predichas
reales = np.argmax(y_test, axis=1)  # Clasificaciones reales de las imágenes
predichos = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo
num_clases = len(data.class_names)  # Número de clases (asegúrate de que coincida)

# Generar la matriz de confusión
conf_matrix = confusion_matrix(reales, predichos)

# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(12, 12))
sns.heatmap(
    conf_matrix,
    annot=False,                     # No anotar los valores en la matriz
    fmt='d',                         # Formato de anotación
    cmap='Blues',                    # Colormap para la visualización
    xticklabels=data.class_names,    # Etiquetas para el eje X
    yticklabels=data.class_names     # Etiquetas para el eje Y
)

# Etiquetas y título
plt.xlabel('Predicho')              # Etiqueta del eje X
plt.ylabel('Real')                  # Etiqueta del eje Y
plt.title('Matriz de confusión')    # Título de la gráfica
plt.show()

* Metricas de precision

In [ ]:
from sklearn.metrics import recall_score, f1_score
import numpy as np
# Calcular las clasificaciones reales y predichas
reales = np.argmax(y_test, axis=1)  # Clasificaciones reales de las imágenes
predichos = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo

# Calcular recall y F1 score de forma micro y macro
recall_micro = recall_score(reales, predichos, average='micro')
f1_micro = f1_score(reales, predichos, average='micro')

recall_macro = recall_score(reales, predichos, average='macro')
f1_macro = f1_score(reales, predichos, average='macro')

# Mostrar los resultados
print(f"Recall Micro: {recall_micro:.4f}")
print(f"F1 Score Micro: {f1_micro:.4f}")
print(f"Recall Macro: {recall_macro:.4f}")
print(f"F1 Score Macro: {f1_macro:.4f}")

## Modelo de redes convolucionales

* Carga de datos

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Especificar el directorio que contiene las imágenes
#directory = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"
directory = "C:/Users/cesar/OneDrive/Escritorio/vehicle_data"

# Cargar los datos como root_directory
data = keras.utils.image_dataset_from_directory(
    directory,               # Directorio que contiene las imágenes
    labels="inferred",       # Inferir las etiquetas de los nombres de las carpetas
    label_mode="int",        # Usar enteros como modo de etiquetas
    color_mode="rgb",        # Cargar imágenes en modo RGB
    batch_size=6,            # Tamaño del lote
    shuffle=True,            # Aleatorizar los datos
    seed=1234                # Semilla para la aleatorización
)


* Division de la base y transformacion de los datos

In [ ]:
# Importar las librerías necesarias
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# Convertir datos a numpy arrays
# (esto puede variar dependiendo de cómo cargaste los datos)
images, labels = [], []  # Listas para almacenar imágenes y etiquetas

# Recorrer los lotes de datos y agregar a las listas
for image_batch, label_batch in data:
    images.extend(image_batch.numpy())  # Convertir y agregar imágenes
    labels.extend(label_batch.numpy())   # Convertir y agregar etiquetas

# Convertir listas a arrays de numpy
images = np.array(images)
labels = np.array(labels)

# Estratificación y división de datos
# X contendrá las imágenes y y contendrá las etiquetas
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.3, random_state=42, stratify=labels
)

# Dividir el conjunto de prueba en validación y prueba
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

# Mostrar la cantidad de datos en cada conjunto
print(len(y_train), len(y_val), len(y_test))

# Normalizar los píxeles entre 0 y 1
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Codificación one-hot para las etiquetas
num_classes = len(data.class_names)  # Número de clases
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

* Verificar conteos por division, etc.

In [ ]:
# Imprimir las formas de los conjuntos de datos
print("Forma del conjunto de entrenamiento:", X_train.shape)
print("Forma del conjunto de validación:", X_val.shape)

# Mostrar los nombres de las clases en el conjunto de datos
print("Nombres de las clases:", data.class_names)
print("Número de categorías:", len(data.class_names))

# Imprimir los valores mínimo y máximo en el conjunto de entrenamiento
print("Valor mínimo en X_train:", X_train.min())
print("Valor máximo en X_train:", X_train.max())
# Numero de clases como variable
k = 20
X_train.shape

* Modelo y entrenamiento

In [ ]:
# Importar TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras import layers, models

# Crear modelo secuencial
model = models.Sequential()

# Capa Conv2D: 32 filtros 3x3, activación ReLU,
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Capa Conv2D: 64 filtros 3x3, activación ReLU
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Capa Conv2D: 64 filtros 3x3, activación ReLU
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())

# Capa densa: 64 unidades, activación ReLU
model.add(layers.Dense(64, activation='relu'))

# Capa de salida: 'k' clases, activación softmax
model.add(layers.Dense(k, activation='softmax'))

# Mostrar resumen del modelo
model.summary()

# Compilar el modelo con Adam y una tasa de aprendizaje reducida
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early Stopping: detener el entrenamiento si no mejora después de 10 épocas
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

* Aumentacion de datos

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Se crea una instancia de "image data generator" con los distintos tipos de aumentación que se quieran
datagen = ImageDataGenerator(
    rotation_range=15,          # Random rotation
    width_shift_range=0.1,      # Random horizontal shift
    height_shift_range=0.1,     # Random vertical shift
    horizontal_flip=True,       # Random horizontal flip
    zoom_range=0.1              # Random zoom
)

# Se ajusta el generador a los datos de entrenamiento
datagen.fit(X_train)


* Entrenamiento

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

# Se usa datagen.flow() para aumentar los datos
train_generator = datagen.flow(X_train, y_train, batch_size=64)

# Compilación del modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# Entrenar el modelo
history = model.fit(train_generator,  # Datos de entrenamiento
                    epochs=100,  # Número máximo de épocas
                    batch_size=32,  # Tamaño del lote
                    validation_data=(X_val, y_val),  # Datos de validación
                    callbacks=[early_stopping])  # Callback de early stopping

* Evaluacion del modelo

In [ ]:
basic_model_eval = model.evaluate(X_test, y_test)
print(f"Modelo básico: error y accuracy {basic_model_eval}")

* Guardar modelo

In [ ]:
# Guardar el modelo
#model.save('modelConvo_1.keras')  # Mejor usar el formato .keras en lugar de .h5

# Cargar el modelo
#from tensorflow.keras.models import load_model
#model = load_model('modelConvo_1.keras')

* Evaluacion grafica

In [ ]:
# Importar las librerías necesarias
import matplotlib.pyplot as plt  # Para graficar
import tensorflow as tf          # Para manejar el historial del entrenamiento

# Acceder a la precisión de entrenamiento y validación desde el historial
train_accuracy = history.history['accuracy']    # Precisión de entrenamiento
val_accuracy = history.history['val_accuracy']  # Precisión de validación

# Acceder a la pérdida de entrenamiento y validación desde el historial
train_loss = history.history['loss']         # Pérdida de entrenamiento
val_loss = history.history['val_loss']       # Pérdida de validación

# Configuración de la figura con 1 fila y 2 columnas
fig, axs = plt.subplots(1, 2, figsize=(16, 6))  # Ajusta el tamaño según lo necesites

# Graficar precisión de entrenamiento y validación
axs[0].plot(train_accuracy, label='Entrenamiento', color="blue")    # Precisión de entrenamiento
axs[0].plot(val_accuracy, label='Validación', color="red")          # Precisión de validación
axs[0].set_title('Accuracy de entrenamiento por epoch')  # Título del gráfico
axs[0].set_xlabel('Epoch')                               # Etiqueta del eje X
axs[0].set_ylabel('Precisión')                           # Etiqueta del eje Y
axs[0].set_ylim(0, 1)
axs[0].legend()                                          # Mostrar la leyenda
axs[0].grid(True)                                        # Activar la cuadrícula

# Graficar pérdida de entrenamiento y validación
axs[1].plot(train_loss, label='Entrenamiento', color="blue")    # Pérdida de entrenamiento
axs[1].plot(val_loss, label='Validación', color="red")          # Pérdida de validación
axs[1].set_title('Error de entrenamiento por epoch')   # Título del gráfico
axs[1].set_xlabel('Epoch')                             # Etiqueta del eje X
axs[1].set_ylabel('Error')                             # Etiqueta del eje Y
axs[1].legend()                                        # Mostrar la leyenda
axs[1].grid(True)                                      # Activar la cuadrícula

plt.tight_layout()                                     # Ajustar el diseño para evitar solapamientos
plt.show()                                             # Mostrar los gráficos

* Matriz de confucion

In [ ]:
# Importar las librerías necesarias
import numpy as np                         # Para operaciones numéricas y manipulación de arrays
from sklearn.metrics import confusion_matrix  # Para generar la matriz de confusión
import seaborn as sns                      # Para visualizar la matriz de confusión
import matplotlib.pyplot as plt            # Para visualización de gráficos

# Calcular las clasificaciones reales y predichas
reales = np.argmax(y_test, axis=1)  # Clasificaciones reales de las imágenes
predichos = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo
num_clases = len(data.class_names)  # Número de clases (asegúrate de que coincida)

# Generar la matriz de confusión
conf_matrix = confusion_matrix(reales, predichos)

# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(12, 12))
sns.heatmap(
    conf_matrix,
    annot=False,                     # No anotar los valores en la matriz
    fmt='d',                         # Formato de anotación
    cmap='Blues',                    # Colormap para la visualización
    xticklabels=data.class_names,    # Etiquetas para el eje X
    yticklabels=data.class_names     # Etiquetas para el eje Y
)

# Etiquetas y título
plt.xlabel('Predicho')              # Etiqueta del eje X
plt.ylabel('Real')                  # Etiqueta del eje Y
plt.title('Matriz de confusión')    # Título de la gráfica
plt.show()

* Prueba de clasificaciones

In [ ]:
# Importar las librerías necesarias
import numpy as np           # Para operaciones numéricas y manipulación de arrays
import matplotlib.pyplot as plt  # Para visualización de imágenes

# Mostrar 10 imágenes con sus clasificaciones reales y predichas
n = 10  # Número de imágenes a mostrar
fig, ax = plt.subplots(1, n, figsize=(15, 15))  # Ajustamos el tamaño del gráfico

for i in range(n):
    # Obtener la clasificación real y la predicción del modelo
    real = np.argmax(y_test[i])  # Clasificación real de la imagen

    # Como las imágenes no están aplanadas, no hacemos reshape
    img = X_test[i]  # Tomamos la imagen directamente

    # Realizar la predicción del modelo
    prediction = model.predict(img.reshape(1, img.shape[0], img.shape[1], img.shape[2]))  # Predicción del modelo
    prediction = np.argmax(prediction)  # Obtener el índice de la clase predicha

    # Mostrar la imagen
    ax[i].imshow(img)  # Mostrar la imagen en su forma original
    ax[i].axis("off")  # Ocultar los ejes
    ax[i].set_title(f"Real: {data.class_names[real]}, \nClasificado: {data.class_names[prediction]}", fontsize=7)  # Título

plt.show()  # Mostrar la figura con las imágenes

* SHAP

In [ ]:
import numpy as np
import shap
import matplotlib.pyplot as plt

# Supongamos que X_test es tu conjunto de datos de prueba y y_test son las etiquetas correspondientes.

# Step 1: Define a function that returns the model's predictions for all classes
def f(x):
    return model(x)

# Step 2: Create a masker for the images
masker_blur = shap.maskers.Image("blur(32,32)", X_test[0].shape)

# Step 3: Create an Explainer object
explainer = shap.Explainer(f, masker_blur, output_names=[f"Clase {i}" for i in range(20)])

# Step 4: Loop over a subset of images for SHAP explanation
for ind in range(min(5, len(X_test))):  # Cambia 5 por el número de ejemplos que quieras visualizar
    shap_values_ = explainer(X_test[[ind]], max_evals=5000, batch_size=50)

    # Get the class probabilities from the model
    class_probs = model.predict(X_test[[ind]])[0]
    rounded_probs = np.round(class_probs, 2)  # Redondear las probabilidades a dos decimales

    # Print the predicted probabilities for each class
    print(f"Probabilidades para la imagen {ind}:")
    for i in range(len(rounded_probs)):
        print(f"Clase {i}: {rounded_probs[i]:.2f}")  # Asegurarse de imprimir con dos decimales

    # Plot SHAP values
    shap.image_plot(shap_values_, labels=[f"Clase {i}" for i in range(20)], show=False)

    # Set title for the plot with the actual class
    plt.title(f"Clase real: {np.argmax(y_test[[ind]][0])}", fontsize=14)
    plt.show()


* Metricas de precision

In [ ]:
from sklearn.metrics import recall_score, f1_score

# Calcular las clasificaciones reales y predichas
reales = np.argmax(y_test, axis=1)  # Clasificaciones reales de las imágenes
predichos = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo

# Calcular recall y F1 score de forma micro y macro
recall_micro = recall_score(reales, predichos, average='micro')
f1_micro = f1_score(reales, predichos, average='micro')

recall_macro = recall_score(reales, predichos, average='macro')
f1_macro = f1_score(reales, predichos, average='macro')

# Mostrar los resultados
print(f"Recall Micro: {recall_micro:.4f}")
print(f"F1 Score Micro: {f1_micro:.4f}")
print(f"Recall Macro: {recall_macro:.4f}")
print(f"F1 Score Macro: {f1_macro:.4f}")

# Modelo preentrenado

* Carga de datos

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Especificar el directorio que contiene las imágenes
#directory = "/content/drive/MyDrive/Ciencia de datos/Proyecto/vehicle_data"
directory = "C:/Users/cesar/OneDrive/Escritorio/vehicle_data"

# Cargar los datos como root_directory
data = keras.utils.image_dataset_from_directory(
    directory,               # Directorio que contiene las imágenes
    labels="inferred",       # Inferir las etiquetas de los nombres de las carpetas
    label_mode="int",        # Usar enteros como modo de etiquetas
    color_mode="rgb",        # Cargar imágenes en modo RGB
    batch_size=6,            # Tamaño del lote
    shuffle=True,            # Aleatorizar los datos
    seed=1234                # Semilla para la aleatorización
)

* Transformaciones

In [ ]:
# Importar las librerías necesarias
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

# Convertir datos a numpy arrays
# (esto puede variar dependiendo de cómo cargaste los datos)
images, labels = [], []  # Listas para almacenar imágenes y etiquetas

# Recorrer los lotes de datos y agregar a las listas
for image_batch, label_batch in data:
    images.extend(image_batch.numpy())  # Convertir y agregar imágenes
    labels.extend(label_batch.numpy())   # Convertir y agregar etiquetas

# Convertir listas a arrays de numpy
images = np.array(images)
labels = np.array(labels)

# Estratificación y división de datos
# X contendrá las imágenes y y contendrá las etiquetas
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.3, random_state=42, stratify=labels
)

# Dividir el conjunto de prueba en validación y prueba
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42, stratify=y_test
)

k=20
# Mostrar la cantidad de datos en cada conjunto
print(len(y_train), len(y_val), len(y_test))

# Normalizar los píxeles entre 0 y 1
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Codificación one-hot para las etiquetas
num_classes = len(data.class_names)  # Número de clases
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

* Modelo y entrenamiento

In [ ]:
from tensorflow.keras.applications import ResNet50  # Importa el modelo ResNet50 preentrenado
from tensorflow.keras.models import Model  # Importa la clase Model para crear un modelo personalizado
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D  # Importa capas Dense y GlobalAveragePooling2D

# Pre-entrenado ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))  # Carga ResNet50

# Capas personalizadas para clasificación
x = base_model.output  # Obtiene la salida del modelo base
x = GlobalAveragePooling2D()(x)  # Aplica un Pooling global para reducir dimensionalidad
x = Dense(256, activation='relu')(x)  # Añade una capa densa de 256 neuronas con activación ReLU
predictions = Dense(k, activation='softmax')(x)  # Crea la capa de salida con activación softmax para k clases
model = Model(inputs=base_model.input, outputs=predictions)  # Define el modelo combinando el modelo base y las capas añadidas

for layer in base_model.layers:
    layer.trainable = False  # Desactiva el entrenamiento de las capas del modelo base

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compila el modelo con optimizador Adam y entropía cruzada categórica

* Entrenar modelo

In [ ]:
history = model.fit(X_train,y_train, epochs=40, batch_size=64, validation_data=(X_val, y_val))

* Evaluacion del modelo

In [ ]:
basic_model_eval = model.evaluate(X_test, y_test)
print(f"Modelo básico: error y accuracy {basic_model_eval}")

* Guardar modelo

In [ ]:
# Guardar modelo
#model.save('model_preentrenado.h5')
# Volver a cargar en el ambiente un modelo guardado
#from tensorflow.keras.models import load_model
#model = load_model('model_preentrenado.h5')

* Graficacion

In [ ]:
# Importar las librerías necesarias
import matplotlib.pyplot as plt  # Para graficar
import tensorflow as tf          # Para manejar el historial del entrenamiento

# Acceder a la precisión de entrenamiento y validación desde el historial
train_accuracy = history.history['accuracy']    # Precisión de entrenamiento
val_accuracy = history.history['val_accuracy']  # Precisión de validación

# Acceder a la pérdida de entrenamiento y validación desde el historial
train_loss = history.history['loss']         # Pérdida de entrenamiento
val_loss = history.history['val_loss']       # Pérdida de validación

# Configuración de la figura con 1 fila y 2 columnas
fig, axs = plt.subplots(1, 2, figsize=(16, 6))  # Ajusta el tamaño según lo necesites

# Graficar precisión de entrenamiento y validación
axs[0].plot(train_accuracy, label='Entrenamiento', color="blue")    # Precisión de entrenamiento
axs[0].plot(val_accuracy, label='Validación', color="red")          # Precisión de validación
axs[0].set_title('Accuracy de entrenamiento por epoch')  # Título del gráfico
axs[0].set_xlabel('Epoch')                               # Etiqueta del eje X
axs[0].set_ylabel('Precisión')                           # Etiqueta del eje Y
axs[0].set_ylim(0, 1)
axs[0].legend()                                          # Mostrar la leyenda
axs[0].grid(True)                                        # Activar la cuadrícula

# Graficar pérdida de entrenamiento y validación
axs[1].plot(train_loss, label='Entrenamiento', color="blue")    # Pérdida de entrenamiento
axs[1].plot(val_loss, label='Validación', color="red")          # Pérdida de validación
axs[1].set_title('Error de entrenamiento por epoch')   # Título del gráfico
axs[1].set_xlabel('Epoch')                             # Etiqueta del eje X
axs[1].set_ylabel('Error')                             # Etiqueta del eje Y
axs[1].legend()                                        # Mostrar la leyenda
axs[1].grid(True)                                      # Activar la cuadrícula

plt.tight_layout()                                     # Ajustar el diseño para evitar solapamientos
plt.show()                                             # Mostrar los gráficos

* Matriz de confucion

In [ ]:
# Importar las librerías necesarias
import numpy as np                         # Para operaciones numéricas y manipulación de arrays
from sklearn.metrics import confusion_matrix  # Para generar la matriz de confusión
import seaborn as sns                      # Para visualizar la matriz de confusión
import matplotlib.pyplot as plt            # Para visualización de gráficos

# Calcular las clasificaciones reales y predichas
reales = np.argmax(y_test, axis=1)  # Clasificaciones reales de las imágenes
predichos = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo
num_clases = len(data.class_names)  # Número de clases (asegúrate de que coincida)

# Generar la matriz de confusión
conf_matrix = confusion_matrix(reales, predichos)

# Graficar la matriz de confusión usando seaborn
plt.figure(figsize=(12, 12))
sns.heatmap(
    conf_matrix,
    annot=False,                     # No anotar los valores en la matriz
    fmt='d',                         # Formato de anotación
    cmap='Blues',                    # Colormap para la visualización
    xticklabels=data.class_names,    # Etiquetas para el eje X
    yticklabels=data.class_names     # Etiquetas para el eje Y
)

# Etiquetas y título
plt.xlabel('Predicho')              # Etiqueta del eje X
plt.ylabel('Real')                  # Etiqueta del eje Y
plt.title('Matriz de confusión')    # Título de la gráfica
plt.show()

* Prueba de clasificaciones

In [ ]:
# Importar las librerías necesarias
import numpy as np           # Para operaciones numéricas y manipulación de arrays
import matplotlib.pyplot as plt  # Para visualización de imágenes

# Mostrar 10 imágenes con sus clasificaciones reales y predichas
n = 10  # Número de imágenes a mostrar
fig, ax = plt.subplots(1, n, figsize=(15, 15))  # Ajustamos el tamaño del gráfico

for i in range(n):
    # Obtener la clasificación real y la predicción del modelo
    real = np.argmax(y_test[i])  # Clasificación real de la imagen

    # Como las imágenes no están aplanadas, no hacemos reshape
    img = X_test[i]  # Tomamos la imagen directamente

    # Realizar la predicción del modelo
    prediction = model.predict(img.reshape(1, img.shape[0], img.shape[1], img.shape[2]))  # Predicción del modelo
    prediction = np.argmax(prediction)  # Obtener el índice de la clase predicha

    # Mostrar la imagen
    ax[i].imshow(img)  # Mostrar la imagen en su forma original
    ax[i].axis("off")  # Ocultar los ejes
    ax[i].set_title(f"Real: {data.class_names[real]}, \nClasificado: {data.class_names[prediction]}", fontsize=7)  # Título

plt.show()  # Mostrar la figura con las imágenes

* Metricas

In [ ]:
from sklearn.metrics import recall_score, f1_score

# Calcular las clasificaciones reales y predichas
reales = np.argmax(y_test, axis=1)  # Clasificaciones reales de las imágenes
predichos = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo

# Calcular recall y F1 score de forma micro y macro
recall_micro = recall_score(reales, predichos, average='micro')
f1_micro = f1_score(reales, predichos, average='micro')

recall_macro = recall_score(reales, predichos, average='macro')
f1_macro = f1_score(reales, predichos, average='macro')

# Mostrar los resultados
print(f"Recall Micro: {recall_micro:.4f}")
print(f"F1 Score Micro: {f1_micro:.4f}")
print(f"Recall Macro: {recall_macro:.4f}")
print(f"F1 Score Macro: {f1_macro:.4f}")